In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import LinearSVC, SVC
from sklearn import linear_model, tree, neighbors
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import f1_score, accuracy_score
from sklearn import preprocessing
from nltk import pos_tag
import time
from analyze_words import get_final_df
import joblib

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/rhedintzaaudryna/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
from analyze_words import *

In [3]:
def applyModels(model, x_train, y_train):
    print(model)
    model.fit(x_train, y_train)
    return model


def predictModel(model, x_test):
    prediction = model.predict(x_test)
    return prediction

In [5]:
%%time
df, idf, stop_words = get_final_df('test_data/merged_data.csv')

CPU times: user 2min 32s, sys: 31.8 s, total: 3min 3s
Wall time: 3min 14s


In [9]:
df[df.Rating == 3]

,gross,bloody,burgers,three,heady,beers,blaring,music,ears,still,...,tour very,enjoyable pricing,pricing at,at seems,seems little,steep group,group tour,tour end,end very,Rating
4000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
4001,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
4002,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
4003,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
4004,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
5996,0.0,0.0,1.457128,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
5997,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
5998,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3


In [6]:
def transformFeatureSelection(model, x):
    return model.transform(x)

def applyFeatureSelection(model, x_train, y_train):
    model = model.fit(x_train, y_train)
    return model

In [7]:
x_train, x_test, y_train, y_test = train_test_split(df.drop("Rating", axis=1),df.Rating,test_size=0.2)
model = linear_model.SGDClassifier()

In [8]:
trained_model = applyModels(model, x_train, y_train)

SGDClassifier()


In [9]:
feature_selection_model = SelectFromModel(trained_model)
trained_feature_selection_model = applyFeatureSelection(feature_selection_model,
                                                        x_train, y_train)
x_train = transformFeatureSelection(trained_feature_selection_model,
                                    x_train)
x_test = transformFeatureSelection(trained_feature_selection_model,
                                   x_test)

In [10]:
x_train

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.36605709, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [11]:
feature_idx = trained_feature_selection_model.get_support()

In [12]:
feature_name = df.drop("Rating", axis=1).columns[feature_idx]

In [13]:
final_model = applyModels(model, x_train, y_train)

SGDClassifier()


In [39]:
prediction = predictModel(final_model, x_test)

In [41]:
prediction[:50]

array([5, 4, 4, 2, 5, 3, 5, 3, 4, 1, 4, 5, 3, 2, 5, 1, 3, 1, 1, 2, 5, 4,
       5, 5, 2, 3, 5, 1, 2, 4, 5, 1, 3, 3, 2, 1, 4, 4, 2, 2, 3, 3, 5, 5,
       1, 3, 2, 3, 1, 2])

In [15]:
np.mean(prediction == y_test)

0.998

In [16]:
import json

In [17]:
comb = {"ngram": 2, "lemmatize": True, "stop_word": 20}

In [18]:
with open('columns.json', 'w') as f:
        json.dump(list(feature_name), f)
with open('idf.json', 'w') as f:
    json.dump(idf, f)
with open('combination.json', 'w') as f:
    json.dump(comb, f)
with open('stop_words.json', 'w') as f:
    json.dump(stop_words, f)

In [19]:
from main import process_input

In [61]:
%%time
x_array = process_input("""Due to COVID we decided to enjoy an at home Valentine's Day dinner via a meal kit. Sadly it was a mistake. It was not a meal kit for 4 as described online and in their instructions, but rather 2.

Total Score: 2.4/5
Ease of Pickup: 2/5 Naperville-They sat in their truck playing on their phones waiting to set up and made us wait 20 minutes. Finally someone demanded theirs so we got ours too.
Quality of ingredients: 5/5
Quality of prepared food: 3/5 - it was akin to take and bake pizza.
Accuracy of kit: 1/5 - Only received pig face for two with a total weight of 9 ounces and  two chocolate chip cookies. The empanadas and green beans were as described.
Value: 1/5

I contacted the restaurant. They told me the ingredients I received were correct except I should have received 4 cookies. They also said it had changed. When? Before or after The Girl and The Goat printed the instructions, advertised the meal kit, or I paid $133 for a 4 person dinner. To their credit they did refund my money. Such a disappointing experience from one of my former favorite restaurants""")

CPU times: user 46.8 ms, sys: 4.56 ms, total: 51.3 ms
Wall time: 51.9 ms


In [62]:
feature_name[x_array.nonzero()]

Index(['got', 'so', 'or', 'after', 'two', 'have', 'received', 'refund',
       'waiting', 'an',
       ...
       'phones', 'their phones', 'covid decided', 'credit', 'face', 'accuracy',
       'so got', 'theirs', 'prepared food', 'score'],
      dtype='object', length=106)

In [63]:
len(x_array)

26360

In [64]:
x_test.shape

(2000, 26360)

In [65]:
prediction = predictModel(final_model, [x_array])

In [66]:
prediction

array([1])

In [42]:
def forward_selection(df):
    '''
    Given a dataset with P predictor variables, uses forward selection to
    select models for every value of K between 1 and P.

    Inputs:
        dataset: (DataSet object) a dataset

    Returns:
        A list (of length P) of Model objects. The first element is the
        model where K=1, the second element is the model where K=2, and so on.
    '''

    models = []
    pred_vars_avail = dataset.pred_vars[:]
    pred_vars_used = []
    num_vars = len(dataset.pred_vars)

    for _ in range(num_vars):
        max_R2 = 0
        max_model = None
        max_variable = None
        for var in pred_vars_avail:
            model = Model(dataset, pred_vars_used + [var])
            if model.R2 > max_R2:
                max_model = model
                max_R2 = model.R2
                max_variable = var
        pred_vars_used.append(max_variable)
        pred_vars_avail.remove(max_variable)
        models.append(max_model)

    return models